<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e10d899869550379354e83513d1ff12e061939bd6babdfd572c6e3778823cf62
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 14:40:56
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 85.49 K (0.61%)
Current PnL: -17.96 L (-12.08%)
CY Booked + Current PnL: -6.93 L (-4.66%)
-------------------
Total profit:  2.00 L
Total loss:  -19.97 L
-------------------
Total Booked + Current PnL: 20.05 L (16.47%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.81%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.61 L (54.94%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.87,63.0,H-LC,11.20,174196.0,18019.0,8083.0,1.18,11.54,4.64,16.71,22.0,2.23,1.34,30.99,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.76,63.0,M-SC,5.57,88654.0,-12123.0,12217.0,-0.29,-12.03,13.78,0.09,245.0,-0.99,0.68,16.47,OX40N,NTT,DURABLES
62,SBIN,863.00,-12.30,55.0,M-LC,4.27,216386.0,14906.0,12312.0,0.95,7.40,5.69,13.51,60.0,1.21,1.66,20.93,XY25,NTT,BANKS
41,ITC,452.00,-37.50,53.0,H-LC,3.27,201395.0,1257.0,19636.0,0.44,0.63,9.75,10.44,4.0,0.06,1.55,6.23,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.83,110696.0,10192.0,20446.0,1.08,10.14,18.47,30.49,79.0,0.50,0.85,45.83,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,61.0,M-MC,7.06,223447.0,-1515.0,169172.0,0.24,-0.67,75.71,74.53,192.0,-0.01,1.72,31.36,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,39.0,H-SC,6.41,191068.0,62.0,57645.0,-0.25,0.03,30.17,30.21,157.0,0.00,1.47,20.92,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-20.77,43.0,H-MC,1.99,136414.0,626.0,42125.0,0.91,0.46,30.88,31.48,107.0,0.01,1.05,12.64,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-1.08,57.0,H-MC,1.76,245388.0,-2343.0,77395.0,-0.89,-0.95,31.54,30.30,92.0,-0.03,1.88,12.22,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.11,45.0,H-MC,3.49,137381.0,1248.0,29880.0,1.18,0.92,21.75,22.86,91.0,0.04,1.05,13.65,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,JIOFIN,387.00,-2.04,41.0,H-LC,13.75,216650.0,-756.0,54249.0,0.50,-0.35,25.04,24.61,48.0,-0.01,1.66,54.28,XY24,BTT,FINANCE
83,VBL,671.64,-15.52,41.0,H-LC,6.38,304454.0,-11388.0,124065.0,0.57,-3.61,40.75,35.67,5.0,-0.09,2.34,9.75,X40N,ATH,FMCG
56,RELIANCE,1533.00,-14.41,46.0,H-LC,6.49,214640.0,4274.0,24512.0,-0.02,2.03,11.42,13.68,37.0,0.17,1.65,18.89,XY25,NTT,REFINERIES
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.26,172445.0,-9314.0,44560.0,0.15,-5.12,25.84,19.39,86.0,-0.21,1.32,2.61,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,229832.0,-12092.0,75362.0,2.58,-5.00,32.79,26.15,8.0,-0.16,1.76,8.61,X40,ATH,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,59.0,H-LC,1.33,238905.0,-7246.0,86460.0,2.38,-2.94,36.19,32.18,16.0,-0.08,1.83,32.04,X200,ATH,IT
50,NESTLEIND,1377.00,-7.71,62.0,H-LC,2.34,284099.0,18673.0,38126.0,0.99,7.04,13.42,21.40,11.0,0.49,2.18,14.15,XY25,NTT,FMCG
41,ITC,452.00,-37.50,53.0,H-LC,3.27,201395.0,1257.0,19636.0,0.44,0.63,9.75,10.44,4.0,0.06,1.55,6.23,X40,NTT,FMCG
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.94,213980.0,9480.0,31926.0,0.66,4.64,14.92,20.25,10.0,0.30,1.64,13.77,X40N,BTT,PHARMA
83,VBL,671.64,-15.52,41.0,H-LC,6.38,304454.0,-11388.0,124065.0,0.57,-3.61,40.75,35.67,5.0,-0.09,2.34,9.75,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,-18.00,61.0,H-MC,8.20,81809.0,-20843.0,46443.0,5.22,-20.30,56.77,24.94,98.0,-0.45,0.63,24.36,OX40N,NTT,IT
76,TRIDENT,48.00,-6.66,60.0,M-SC,5.97,72667.0,-19588.0,45889.0,4.66,-21.23,63.15,28.51,224.0,-0.43,0.56,24.34,XR,NTT,TEXTILES
79,UNIONBANK,163.00,-16.28,52.0,M-LC,8.96,150436.0,9596.0,34901.0,3.98,6.81,23.20,31.59,66.0,0.27,1.16,35.11,XY24,NTT,BANKS
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.52,180315.0,507.0,111903.0,3.96,0.28,62.06,62.52,88.0,0.00,1.38,34.46,XR,NTT,BANKS
54,RAJOOENG,143.33,-34.72,39.0,H-SC,10.71,99000.0,-3500.0,44332.0,3.48,-3.41,44.78,39.83,114.0,-0.08,0.76,3.48,AR,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-332.06,60.0,H-SC,8.71,98748.0,-34538.0,119130.0,-2.54,-25.91,120.64,63.47,139.0,-0.29,0.76,65.53,XR,NTT,CHEMICALS
46,LAOPALA,464.00,115.96,52.0,H-SC,3.07,68700.0,-32146.0,57969.0,-2.11,-31.88,84.38,25.61,142.0,-0.55,0.53,27.37,AR,NTT,CERAMICS
65,SIEMENS,7969.85,-2.40,50.0,H-LC,29.43,156825.0,-29270.0,241667.0,-1.68,-15.73,154.10,114.13,15.0,-0.12,1.20,15.49,AR,ATH,ELECTRICAL
44,KANSAINER,340.00,-67.97,55.0,H-SC,2.47,220284.0,-49383.0,85713.0,-1.57,-18.31,38.91,13.47,138.0,-0.58,1.69,11.42,XY24,NTT,PAINTS
81,VAIBHAVGBL,521.00,52.62,61.0,H-SC,10.68,101886.0,-50105.0,124230.0,-1.44,-32.97,121.93,48.77,125.0,-0.40,0.78,18.59,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,71.0,H-SC,12.81,129924.0,-12555.0,32078.0,1.37,-8.81,24.69,13.70,163.0,-0.39,1.00,53.35,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.76,63.0,M-SC,5.57,88654.0,-12123.0,12217.0,-0.29,-12.03,13.78,0.09,245.0,-0.99,0.68,16.47,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,55.0,H-SC,2.47,220284.0,-49383.0,85713.0,-1.57,-18.31,38.91,13.47,138.0,-0.58,1.69,11.42,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,113.17,48.0,M-SC,3.04,139959.0,-34600.0,34640.0,0.09,-19.82,24.75,0.02,221.0,-1.00,1.07,21.74,OX40N,NTT,PAINTS
72,TATAELXSI,9161.0,-18.00,61.0,H-MC,8.20,81809.0,-20843.0,46443.0,5.22,-20.30,56.77,24.94,98.0,-0.45,0.63,24.36,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,51.0,H-LC,13.95,291832.0,-54124.0,177988.0,1.81,-15.64,60.99,35.80,1.0,-0.30,2.24,3.38,X40,BTT,IT
39,INFY,2275.00,-16.37,57.0,H-LC,10.12,326018.0,13130.0,158543.0,1.75,4.20,48.63,54.87,3.0,0.08,2.50,11.12,X40,BTT,IT
41,ITC,452.00,-37.50,53.0,H-LC,3.27,201395.0,1257.0,19636.0,0.44,0.63,9.75,10.44,4.0,0.06,1.55,6.23,X40,NTT,FMCG
83,VBL,671.64,-15.52,41.0,H-LC,6.38,304454.0,-11388.0,124065.0,0.57,-3.61,40.75,35.67,5.0,-0.09,2.34,9.75,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,48.0,H-LC,12.54,247589.0,-14030.0,133252.0,0.51,-5.36,53.82,45.57,7.0,-0.11,1.90,5.35,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.54,189000.0,24330.0,43640.0,0.16,14.78,23.09,41.28,89.0,0.56,1.45,23.94,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,39.0,H-SC,6.41,191068.0,62.0,57645.0,-0.25,0.03,30.17,30.21,157.0,0.00,1.47,20.92,X40N,NTT,FINANCE
84,VOLTAS,1918.49,1.02,59.0,H-MC,3.48,211845.0,20103.0,75925.0,-0.49,10.48,35.84,50.08,99.0,0.26,1.63,17.99,XY25,ATH,AC
24,DABUR,735.00,-2.03,63.0,H-MC,5.59,213194.0,16166.0,73467.0,0.44,8.20,34.46,45.49,102.0,0.22,1.64,22.31,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,5.14,107873.0,5869.0,38058.0,0.19,5.75,35.28,43.06,122.0,0.15,0.83,23.85,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.83,110696.0,10192.0,20446.0,1.08,10.14,18.47,30.49,79.0,0.50,0.85,45.83,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,2.47,102780.0,11282.0,67475.0,-0.06,12.33,65.65,86.07,223.0,0.17,0.79,49.26,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.09,49.0,H-SC,12.14,136586.0,13250.0,120510.0,0.09,10.74,88.23,108.45,119.0,0.11,1.05,35.50,AR,ATH,MISC
86,WIPRO,420.00,-9.71,59.0,M-LC,6.49,158373.0,7428.0,102024.0,2.52,4.92,64.42,72.51,53.0,0.07,1.22,11.24,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.52,180315.0,507.0,111903.0,3.96,0.28,62.06,62.52,88.0,0.00,1.38,34.46,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.17,47.0,H-SC,14.69,89182.0,-11781.0,106314.0,2.18,-11.67,119.21,93.63,148.0,-0.11,0.68,32.83,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,42.13,89664.0,-23885.0,63939.0,0.02,-21.03,71.31,35.27,268.0,-0.37,0.69,119.45,XR,NTT,HOTELS
48,MASFIN,398.61,-16.89,55.0,H-SC,13.00,94515.0,-3465.0,25065.0,-1.27,-3.54,26.52,22.05,152.0,-0.14,0.73,37.38,XR,ATH,FINANCE
82,VALIANTORG,838.00,-332.06,60.0,H-SC,8.71,98748.0,-34538.0,119130.0,-2.54,-25.91,120.64,63.47,139.0,-0.29,0.76,65.53,XR,NTT,CHEMICALS
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.83,110696.0,10192.0,20446.0,1.08,10.14,18.47,30.49,79.0,0.50,0.85,45.83,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,59.0,L-SC,42.13,89664.0,-23885.0,63939.0,0.02,-21.03,71.31,35.27,268.0,-0.37,0.69,119.45,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,3.96,81972.0,-20238.0,113630.0,0.22,-19.80,138.62,91.37,208.0,-0.18,0.63,33.07,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3672.97,61.0,M-SC,7.62,117587.0,-29781.0,81594.0,-1.26,-20.21,69.39,35.16,236.0,-0.36,0.90,20.10,XY24,NTT,AUTO
75,TITAGARH,1548.0,-3.20,64.0,H-SC,7.11,158734.0,-33281.0,112161.0,2.61,-17.33,70.66,41.08,158.0,-0.30,1.22,34.51,XY24,NTT,ENGINEERING
64,SHALBY,327.0,1099.36,76.0,M-SC,2.30,163049.0,-18094.0,63557.0,1.69,-9.99,38.98,25.10,235.0,-0.28,1.25,30.67,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,44.39
2,50,75.08


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.86
LC    35.39
MC    23.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.67
X40      16.06
X40N     13.13
XY25     10.30
XR        9.55
AR        8.34
OX40N     7.76
X200      1.83
MH        1.78
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.82
H-SC    25.00
H-MC    20.65
M-SC    14.20
M-LC     7.46
M-MC     2.76
L-SC     1.66
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.99,-15.16,73.46
FMCG,12.30,-3.67,38.81
FINANCE,9.06,-18.49,61.09
BANKS,7.78,-12.54,60.33
PAINTS,6.13,-15.50,41.36
MISC,6.05,-16.84,83.82
ELECTRICAL,5.82,-8.51,68.31
AC,3.83,2.82,30.09
AUTO,3.47,-15.84,68.08


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2731400.0,22
AR,1183782.0,9
XR,1051004.0,13
X40,873277.0,10
X40N,566787.0,10
OX40N,448006.0,10
XY25,410703.0,7
SR,197122.0,2
X5K,129469.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2769771.0,28
M-SC,1395932.0,17
H-LC,1369155.0,15
H-MC,1154288.0,15
M-LC,408379.0,5
M-MC,319471.0,2
L-SC,244974.0,3
L-MC,59816.0,1
L-LC,39410.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,916257.0,6
M-SC,XY24,813021.0,7
H-SC,AR,649207.0,5
H-LC,X40,582255.0,5
H-SC,XR,503610.0,6
H-MC,XY24,463828.0,4
H-LC,AR,374919.0,2
M-MC,XY24,319471.0,2
H-SC,X40N,249083.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
